Medium post: https://medium.com/@kevin_guo/infrared-spectra-classification-experiment-94457e925549

In [ ]:
# Load IR data and split into train/test

import keras
import random
import numpy as np
from os import listdir
from os.path import isfile, join

np.random.seed(0)
random.seed(0)

data_path = 'data/'

aldehydes = np.array([712, 177, 527, 261, 7860, 6561, 8063, 11552, 7284, 6184, 7359, 12417, 31245, # monofunctionalized
                      756, 7897, 751, # alcohol
                      7860, 10964, 12524, # dicarbonyl 
                      75, 188982, # amino 
                      527448, 18635, 18635, # thiol
                      123114, 13002, 7847, 20138, 643950, 5280971, 9543055]) # sterically strained / conjugation

other_carbonyls = np.array([284, 767, 176, # acids
                            8025, 8857]) # esters

non_carbonyls = np.array([79015, 702, 887, 962, # alcohols
                          10795, 8028, 8029, # ethers
                          6329, 878, 6343, 7848, 6058, 6329, 674, # other heteroatomic functionality
                          313, # inorganic
                          297, 6324, 8252, 6334, 8255, 6326, 6335, 1146])# unfunctionalized (un)saturated aliphatic compounds


files = [f.split('.')[0] for f in listdir(data_path) if isfile(join(data_path, f))]
molecules = list(map(int, filter(lambda f: f != '',files)))

print('Molecules: ', molecules)
print('')

ald_perm = np.random.permutation(aldehydes) # deterministically chosen since random seed is set above
carbonyl_perm = np.random.permutation(other_carbonyls)
other_perm = np.random.permutation(non_carbonyls)

train_mol_ids = list(ald_perm[len(aldehydes) // 5:]) + list(
    carbonyl_perm[len(carbonyl_perm) // 5:]) + list(other_perm[len(other_perm) // 5:])

raw_train_X = [] # list of lists of (peak, intensity)
raw_train_y = [] # 0: non-carbonyl, 1: aldehyde, 2: acid/ester
raw_test_X = []
raw_test_y = []

for mol_id in molecules:
    with open(data_path + str(mol_id) + '.dat') as f:
        mol_data = []
        for line in f:
            peak, intensity = line.split()
            if mol_id in aldehydes:
                y = 1
            elif mol_id in other_carbonyls:
                y = 2
            else:
                y = 0
            mol_data.append((peak, intensity))
        if mol_id in train_mol_ids:
            raw_train_X.append(mol_data)
            raw_train_y.append(y)
        else:
            raw_test_X.append(mol_data)
            raw_test_y.append(y)


/Users/kevin/untitled1/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


ValueError: invalid literal for int() with base 10: 'labels'

In [251]:
import keras
import random
import h5py
import numpy as np
from os import listdir
from os.path import isfile, join

np.random.seed(0)
random.seed(0)

functional_groups = ['unfunctionalized alkane', 'alkene', 'alkyne',
                     'alcohol', 'ether', 'amine',
                     'aldehyde', 'ketone', 'carboxylic acid', 'ester', 'amide']

load_only_monofunctional = True 
group_ester_acid = False 

data_path = 'data/'

molecules = []
labels = []

with open('data/labels.txt', 'r') as f:
    for line in f.readlines():
        nist_id = line.split()[0]
        label = list(map(int, line.split()[1:]))
        if load_only_monofunctional:
            if len(label) > 1:
                continue
            else: label = label[0]
        h5f = h5py.File('data/spectra/' + nist_id + '.h5', 'r')
        wavenumbers = np.array(h5f['x'])
        intensities = np.array(h5f['y'])
        h5f.close()
        mol = []
        for wn, intens in zip(wavenumbers, intensities):
            #if intens > 10:
            mol.append((wn, intens))
        if np.max(intensities) != 100:
            #print('something went wrong...') # TODO: fix this
            continue
        molecules.append(mol)
        labels.append(label)

if group_ester_acid:
    labels = list(map(lambda l: 8 if l == 9 else l, labels))
        
perm = np.random.permutation(np.arange(len(molecules)))

raw_train_X = []
raw_train_y = []
raw_test_X = []
raw_test_y = []

for i in range(len(perm) // 5): # split 80:20
    raw_test_X.append(molecules[i])
    raw_test_y.append(labels[i])
while i < len(perm) - 1:
    i += 1
    raw_train_X.append(molecules[i])
    raw_train_y.append(labels[i])
    
print(len(molecules), 'molecular infrared spectra loaded.')
print(len(raw_train_X), 'training examples.')
print(len(raw_test_X), 'test examples.')

911 molecular infrared spectra loaded.
729 training examples.
182 test examples.


In [193]:
# # Augmenting raw data

# # Data augmentation

# # TODO: introduce small changes in intensity/wavenumber
# # Paper on data augmentation for infrared spectra https://arxiv.org/pdf/1710.01927.pdf

# num_augments = 50 # number of times to augment all data
# intens_noise = 15 # std; absolute difference in intensity
# wn_noise = 10 # std; absolute difference in wavenumbers
# peak_noise = (1/250, (10, 10)) # (probability of a random peak appearing, intensity mean/std)

# unaugmented_train_X = raw_train_X.copy()
# unaugmented_test_X = raw_test_X.copy()

# def augment(x):
#     """ Augment data for the given sample. """
#     aug = []
#     noise = np.random.normal([0, 0], [wn_noise, intens_noise], (len(x), 2))
#     for (noise_wn, noise_intens), (peak, intens) in zip(x, noise):
#         aug.append((peak + noise_wn, intens + noise_intens))
#     new_peaks = np.random.binomial(1, peak_noise[0], 2500).nonzero() + 1000 # only generate from 1000 - 3500 cm-1
#     intensities = np.random.normal(*peak_noise[1], len(new_peaks))
#     for wn, intens in zip(new_peaks, intensities):
#         aug.append((wn, intens))
#     return aug

# for _ in range(num_augments):
#     for x in unaugmented_train_X:
#         raw_train_X.append(augment(x))
#     for x in unaugmented_test_X:
#         raw_test_X.append(augment(x))
        
# print('Total Training Samples: ', len(raw_train_X))
# print('Total Testing Samples: ', len(raw_test_X))

In [252]:
# Vectorizing raw data

# classify_carbonyls = False # if True, will group all carbonyls together
dim = 700 # number of dimensions in vectorization; default is every 5 cm-1

# train_y = keras.utils.to_categorical(map(lambda y: 1 if y == 1 else 0, raw_train_y))
# test_y = keras.utils.to_categorical(map(lambda y: 1 if y == 1 else 0, raw_test_y))
# if classify_carbonyls:
#     train_y = keras.utils.to_categorical(map(lambda y: 1 if y == 1 or y == 2 else 0, raw_train_y))
#     test_y = keras.utils.to_categorical(map(lambda y: 1 if y == 1 or y == 2 else 0, raw_test_y))

train_y = keras.utils.to_categorical(raw_train_y)
test_y = keras.utils.to_categorical(raw_test_y)

step = 3500 / (dim)

def vectorize(peaks):
    """ Converts IR data into vector. Only strongest peak in each interval will be kept. """
    vec = np.zeros(dim)
    for wavenumber, intensity in peaks:
        index = int((wavenumber - 500) // step) # data will range from 500 to 4000
        vec[index] = max(intensity, vec[index])
    return vec

train_X = np.array(list(map(vectorize, raw_train_X)))
test_X = np.array(list(map(vectorize, raw_test_X)))

print(len(train_X) + len(test_X), 'examples successfully vectorized into', dim, 'dimensional vectors.')

911 examples successfully vectorized into 700 dimensional vectors.


In [267]:
# Model construction

from keras import models, layers

convolution_distance = 40 # in wavenumbers
kernel_size = int(convolution_distance // step)

model = models.Sequential()
#model.add(layers.Conv1D(32, kernel_size, activation='relu'))
# model.add(layers.Conv1D(64, convolution_distance // step, activation='relu'))
# model.add(layers.MaxPooling1D(convolution_distance // step))
# model.add(layers.Flatten())
model.add(layers.Dense(600, activation='relu', input_shape = (dim,)))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(300, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(100, activation='relu'))
model.add(layers.Dropout(0.5))
# model.add(layers.Dense(100, activation='relu'))
# model.add(layers.Dropout(0.5))
model.add(layers.Dense(len(train_y[0]), activation='softplus'))

model.compile(loss='categorical_crossentropy', optimizer='adamax', metrics=['categorical_accuracy'])

In [270]:
# Train model
# model.fit(train_X, train_y, batch_size=16, epochs=25, validation_data=(test_X, test_y), shuffle=True) # gets to 67% with adamax
# model.fit(train_X, train_y, batch_size=8, epochs=25, validation_data=(test_X, test_y), shuffle=True) # gets to 65% with adamax
model.fit(train_X, train_y, batch_size=8, epochs=10, validation_data=(test_X, test_y), shuffle=True) 

Train on 729 samples, validate on 182 samples
Epoch 1/10
729/729 [==============================] - 1s 1ms/step - loss: 1.5842 - categorical_accuracy: 0.5075 - val_loss: 1.5564 - val_categorical_accuracy: 0.5220
Epoch 2/10
729/729 [==============================] - 1s 1ms/step - loss: 1.6009 - categorical_accuracy: 0.5350 - val_loss: 1.5707 - val_categorical_accuracy: 0.5165
Epoch 3/10
729/729 [==============================] - 1s 1ms/step - loss: 1.6714 - categorical_accuracy: 0.5487 - val_loss: 1.5050 - val_categorical_accuracy: 0.5165
Epoch 4/10
729/729 [==============================] - 1s 1ms/step - loss: 1.4877 - categorical_accuracy: 0.5144 - val_loss: 1.4561 - val_categorical_accuracy: 0.5440
Epoch 5/10
729/729 [==============================] - 1s 1ms/step - loss: 1.7276 - categorical_accuracy: 0.5638 - val_loss: 1.5156 - val_categorical_accuracy: 0.4670
Epoch 6/10
729/729 [==============================] - 1s 1ms/step - loss: 1.5417 - categorical_accuracy: 0.5405 - val_loss: 

In [250]:
model.save('model.hdf5')

In [144]:
print((1700-500) // step)
print(step * 139 + 500)
train_X[10][140]
train_X[10]

137.0
1716.25


array([  0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,
         0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,
         0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,
         0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,
         0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,
         0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,
         0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,
         0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,
         0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,
         0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,  10.76,
        13.38,  15.75,  21.11,  32.06,  32.78,  27.8 ,  18.72,  11.82,
         0.  ,   0.  ,   0.  ,   0.  ,  13.83,  14.44,  13.16,   0.  ,
         0.  ,   0.  ,   0.  ,   0.  ,  11.34,  12.01,  11.46,   0.  ,
         0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,
      